__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-15 10:20:48 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-15 10:20:48 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-15 10:20:50 | AdsListParser: downloader init
[INFO    ] 2022-09-15 10:20:50 | AvitoParser: init
[INFO    ] 2022-09-15 10:20:57 | AvitoParser: 77 pages for read
[INFO    ] 2022-09-15 10:20:57 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-15 10:21:01 | AdsListParser: read page 1
[INFO    ] 2022-09-15 10:21:06 | AdsListParser: read page 2
[INFO    ] 2022-09-15 10:21:10 | AdsListParser: read page 3
[INFO    ] 2022-09-15 10:21:14 | AdsListParser: read page 4
[INFO    ] 2022-09-15 10:21:20 | AdsListParser: read page 5
[INFO    ] 2022-09-15 10:21:24 | AdsListParser: read page 6
[INFO    ] 2022-09-15 10:21:29 | AdsListParser: read page 7
[INFO    ] 2022-09-15 10:21:34 | AdsListParser: read page 8
[INFO    ] 2022-09-15 10:21:40 | AdsListParser: read page 9
[INFO    ] 2022-09-15 10:21:43 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3808
3532


In [ ]:
df['place']=place

In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [9]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
2603,2441418903,"Объявление «1-к. квартира, 35,4 м², 2/8 эт.» 3 фотографии",8496000,ЖК «Дом на Маячной»,"ул. Маячная, жилой дом","Рассрочка от Застройщика до конца строительства! Без % и переплаты. \nПервый взнос по рассрочке- 30%. \nИпотека с господдержкой и семейная ипотека от 4,5%. \nОбратите внимание на удобную, правильную планировку пространства. \nКвартира в жилом комплексе «Дом на",58,2022-09-15 10:27:26.895333
1099,2457654219,"Объявление «1-к. квартира, 31,6 м², 4/5 эт.» 17 фотографий",3000000,,"с. Штурмовое, ул. Комиссара Морозова, 6","Однокомнатная квартира 31,6 м2, на 4/5 этаже дома. Расположена в г. Севастополь, с. Штурмовое, ул. Комиссара Морозова, д. 6. \n\nЭта квартира — замечательный вариант для тех, кто устал от городской суеты и хочет ежедневно наслаждаться тишиной, любуясь красив",23,2022-09-15 10:27:26.895333
